# プロジェクト - 航空会社AIアシスタント

航空会社「FlightAI」のAIカスタマーサポートアシスタントを作成するプロジェクトです。

この Jupyter Notebook（week2/day4.ipynb）の内容の概要は以下の通り。

主な流れは以下の通りです：

1. **OpenAIやGradioなどのライブラリのインポートと初期化**
   - OpenAI APIキーの読み込み、モデル（gpt-4o-mini など）の設定が行われています。
   - Gradioを使って対話型チャットインターフェースを構築します。

2. **システムメッセージの設定**
   - アシスタントは「1文で丁寧かつ正確に答える」「分からない場合は正直に伝える」などのルールが与えられています。

3. **チャット関数の実装**
   - ユーザーからのメッセージと履歴を元にOpenAIのAPIを呼び出し、返答を返します。

4. **ツール（関数）連携**
   - 「get_ticket_price」という目的地ごとの航空券価格を返す関数を実装しています。
   - LLMがこの関数を必要に応じて呼び出せるような仕組み（ツールの記述や登録）を用意しています。

5. **ツール呼び出し対応のチャット関数**
   - LLMの応答にツール呼び出しが含まれていた場合、自動的に対応する関数を実行し、その結果を基に再度OpenAI APIで回答を生成します。

6. **GradioでのチャットUI起動**
   - 実際にチャットボットをWebインターフェースで動かせるようになっています。

要約すると、「AIカスタマーサポートボットを作成し、ユーザーの質問にAIが答えるだけでなく、必要な場合はプログラムのツール（例：航空券価格取得関数）を自動的に呼び出して、より正確な情報を返す」ことを実践的に学ぶノートブックです。

In [1]:
# import

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# 初期化

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI APIキーが存在し、開始します {openai_api_key[:8]}")
else:
    print("Openai APIキーが設定されていません")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

# 別の方法として、OpenAIの代わりにOllamaを使用したい場合
# Ollamaがローカルで走っていることを確認してください
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

OpenAI APIキーが存在し、開始します sk-proj-


In [3]:
system_message = "あなたはFlightAIという航空会社の親切なアシスタントです。\
1文以内で、簡潔で丁寧な回答をお願いします。常に正確に回答してください。\
わからない場合は、その旨を伝えてください。"

In [4]:
# この関数は、最新のアップデートを利用しているため、私のビデオの機能よりもかなりシンプルに見えます。

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    # コレはログ
    print("「history」は次のとおりです。")
    print(history)
    print("そして「messages」は次のとおりです。")
    print(messages)
    
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


「history」は次のとおりです。
[]
そして「messages」は次のとおりです。
[{'role': 'system', 'content': 'あなたはFlightAIという航空会社の親切なアシスタントです。1文以内で、簡潔で丁寧な回答をお願いします。常に正確に回答してください。わからない場合は、その旨を伝えてください。'}, {'role': 'user', 'content': 'londonに行きたい'}]
「history」は次のとおりです。
[{'role': 'user', 'metadata': None, 'content': 'londonに行きたい', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'ロンドン行きのフライトについて調査いたしますので、出発地と希望の旅行日を教えてください。', 'options': None}]
そして「messages」は次のとおりです。
[{'role': 'system', 'content': 'あなたはFlightAIという航空会社の親切なアシスタントです。1文以内で、簡潔で丁寧な回答をお願いします。常に正確に回答してください。わからない場合は、その旨を伝えてください。'}, {'role': 'user', 'metadata': None, 'content': 'londonに行きたい', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'ロンドン行きのフライトについて調査いたしますので、出発地と希望の旅行日を教えてください。', 'options': None}, {'role': 'user', 'content': 'parisを明日出発'}]
「history」は次のとおりです。
[{'role': 'user', 'metadata': None, 'content': 'londonに行きたい', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'ロンドン行きのフライトについて調査いたしますので、出発地と希望の

## ツール（Function calling）

ツールは、フロンティアLLMSが提供する非常に強力な機能です。

ツールを使用すると、関数を記述し、LLMにその機能をその応答の一部として呼び出すことができます。

ほとんど不気味に聞こえます。マシンでコードを実行する力を与えていますか？まあ、ちょっと。

In [5]:
# 役立つ機能を作成することから始めましょう

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    # 辞書 ticket_prices から、指定された都市の料金を取得。辞書にその都市がなければ "Unknown" を返す。
    return ticket_prices.get(city, "Unknown")

In [6]:
get_ticket_price("Berlin")

Tool get_ticket_price called for Berlin


'$499'

In [7]:
# 関数を記述するために必要な特定の辞書構造があります。
#「Function calling」に関連する「関数の定義」部分
#（price_function → get_ticket_price）

price_function = {
    "name": "get_ticket_price",
    "description": "目的地までの往復航空券の料金を取得します。例えば、顧客から「この都市までの航空券はいくらですか？」と尋ねられたときなど、航空券の料金を知りたいときはいつでもこのメソッドを呼び出します。",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "顧客が旅行したい都市",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": price_function}]

## OpenAIを取得してツールを使用します

OpenAIが「私たちのツールを呼び出す」ことを許可するための厄介なものがいくつかあります

私たちが実際に行っていることは、LLMにツールを実行することを望んでいることを知らせる機会を与えることです。

新しいチャット関数がどのように見えるかは次のとおりです。

In [8]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    # コレはログ
    print("「history」は次のとおりです。")
    print(history)
    print("そして「messages」は次のとおりです。")
    print(messages)
    
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls": # LLMがtools中のprice_functionを使うべきと判断した場合
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [9]:
# その関数をhandle_tool_call：書く必要があります：

def handle_tool_call(message):
    print("responseがtool_callsの場合の「messages」は次のとおりです。")
    print(message)
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    
    # 本来はレイトバインドで呼び出す。
    #price = get_ticket_price(city)
    price = globals()[tool_call.function.name](city)
    
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [10]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


「history」は次のとおりです。
[]
そして「messages」は次のとおりです。
[{'role': 'system', 'content': 'あなたはFlightAIという航空会社の親切なアシスタントです。1文以内で、簡潔で丁寧な回答をお願いします。常に正確に回答してください。わからない場合は、その旨を伝えてください。'}, {'role': 'user', 'content': 'londonに行きたい'}]
responseがtool_callsの場合の「messages」は次のとおりです。
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_r9rDMAEgcT4fGR7WbbP4f8zh', function=Function(arguments='{"destination_city":"london"}', name='get_ticket_price'), type='function')])
Tool get_ticket_price called for london
